In [36]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 16: Permutation Promenade ---</h2><p>You come upon a very unusual sight; a group of programs here appear to be <a href="https://www.youtube.com/watch?v=lyZQPjUT5B4&amp;t=53">dancing</a>.</p>
<p>There are sixteen programs in total, named <code>a</code> through <code>p</code>. They start by standing in a <span title="This is called a 'newline'.">line</span>: <code>a</code> stands in position <code>0</code>, <code>b</code> stands in position <code>1</code>, and so on until <code>p</code>, which stands in position <code>15</code>.</p>
<p>The programs' <em>dance</em> consists of a sequence of <em>dance moves</em>:</p>
<ul>
<li><em>Spin</em>, written <code>sX</code>, makes <code>X</code> programs move from the end to the front, but maintain their order otherwise. (For example, <code>s3</code> on <code>abcde</code> produces <code>cdeab</code>).</li>
<li><em>Exchange</em>, written <code>xA/B</code>, makes the programs at positions <code>A</code> and <code>B</code> swap places.</li>
<li><em>Partner</em>, written <code>pA/B</code>, makes the programs named <code>A</code> and <code>B</code> swap places.</li>
</ul>
<p>For example, with only five programs standing in a line (<code>abcde</code>), they could do the following dance:</p>
<ul>
<li><code>s1</code>, a spin of size <code>1</code>: <code>eabcd</code>.</li>
<li><code>x3/4</code>, swapping the last two programs: <code>eabdc</code>.</li>
<li><code>pe/b</code>, swapping programs <code>e</code> and <code>b</code>: <code>baedc</code>.</li>
</ul>
<p>After finishing their dance, the programs end up in order <code>baedc</code>.</p><p>
</p><p>You watch the dance for a while and record their dance moves (your puzzle input). <em>In what order are the programs standing</em> after their dance?</p>
</article>


<link href="style.css" rel="stylesheet"></link>


In [37]:
from string import ascii_lowercase


example = "s1,x3/4,pe/b"


def one_step_dance(pos, ops, n):
    for op in ops:
        if op.startswith("s"):
            d = -int(op[1:]) % n
            pos = pos[d:] + pos[:d]
        elif op.startswith("x"):
            fr, to = map(int, op[1:].split("/"))
            pos[fr], pos[to] = pos[to], pos[fr]
        else:
            fr, to = map(pos.index, op[1:].split("/"))
            pos[fr], pos[to] = pos[to], pos[fr]
    return pos


def dancing(n: int, s: str) -> str:
    ops = s.split(",")
    pos = list(ascii_lowercase[:n])
    pos = one_step_dance(pos, ops, n)
    return "".join(pos)


print(f"Example: {dancing(5, example)}, should be baedc")

Example: baedc, should be baedc


In [38]:
with open("../input/day16.txt") as f:
    puzzle = f.read()

print(f"Part I: {dancing(16, puzzle)}")

Part I: glnacbhedpfjkiom


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>glnacbhedpfjkiom</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you're starting to get a feel for the dance moves, you turn your attention to <em>the dance as a whole</em>.</p>
<p>Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of <em>one billion</em> (<code>1000000000</code>) times.</p>
<p>In the example above, their second dance would <em>begin</em> with the order <code>baedc</code>, and use the same dance moves:</p>
<ul>
<li><code>s1</code>, a spin of size <code>1</code>: <code>cbaed</code>.</li>
<li><code>x3/4</code>, swapping the last two programs: <code>cbade</code>.</li>
<li><code>pe/b</code>, swapping programs <code>e</code> and <code>b</code>: <code>ceadb</code>.</li>
</ul>
<p><em>In what order are the programs standing</em> after their billion dances?</p>
</article>

</main>


In [43]:
def detect_cycles(pos: list[str], ops: list[str], repeat: int) -> tuple[int, int]:
    cycles = {"".join(pos): 0}
    n, step = len(pos), 0
    for _ in range(repeat):
        step += 1
        pos = one_step_dance(pos, ops, n)
        s = "".join(pos)
        if s in cycles:
            return cycles[s], step
        cycles[s] = step
    return 0, -1


def dancing_cycles(pos: list[str], ops: list[str], repeat: int) -> str:
    n = len(pos)
    for _ in range(repeat):
        pos = one_step_dance(pos, ops, n)
    return pos


def dancing_II(n: int, s: str, repeat=1) -> str:
    ops = s.split(",")
    fr, to = detect_cycles(list(ascii_lowercase[:n]), ops, repeat)
    pos = dancing_cycles(list(ascii_lowercase[:n]), ops, fr)
    return "".join(dancing_cycles(pos, ops, (repeat - fr) % (to - fr)))


print(f"Example: {dancing_II(5, example, 2)}, should be ceadb")
print(f"Example: {dancing_II(5, example, 1_000_000_000)}, should be abcde")

Example: abcde, should be ceadb
Example: abcde, should be abcde


In [44]:
print(f"Part II: {dancing_II(16, puzzle, 1_000_000_000)}")

Part II: fmpanloehgkdcbji


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>fmpanloehgkdcbji</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
